In [1]:
import sys
sys.path.append('../../')
from hydroDL import master, utils
from hydroDL.data import camels
from hydroDL.master import default
from hydroDL.master.master import loadModel
from hydroDL.model import rnn, crit, train


import os
import numpy as np
import torch
from collections import OrderedDict
import random
import json
import datetime as dt

## fix the random seeds for reproducibility
randomseed = 111111
random.seed(randomseed)
torch.manual_seed(randomseed)
np.random.seed(randomseed)
torch.cuda.manual_seed(randomseed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## GPU setting
# which GPU to use when having multiple
traingpuid = 0
# torch.cuda.set_device(traingpuid)

## Setting training options here
PUOpt = 0
# PUOpt values and explanations:
# 0: train and test on ALL basins;
# 1 for PUB spatial test, randomly hold out basins;
# 2 for PUR spatial test, hold out a continuous region;
buffOpt = 1
# buffOpt defines the warm-up option for the first year of training forcing data
# 0: do nothing, the first year forcing would only be used to warm up the next year;
# 1: repeat first year forcing to warm up the first year;
# 2: load one more year forcing to warm up the first year
TDOpt = True
# TDOpt, True as using dynamic parameters and False as using static parameters
forType = 'daymet'
# for Type defines which forcing in CAMELS to use: 'daymet', 'nldas', 'maurer'


# used for multimodel ensembling.
prcp_loss_factor = 23
smooth_loss_factor = 0

## Set hyperparameters
EPOCH = 50 # total epoches to train the mode
BATCH_SIZE = 100
RHO = 365
HIDDENSIZE = 256
saveEPOCH = 10
Ttrain = [19801001, 19951001] # Training period
# Ttrain = [19891001, 19991001] # PUB/PUR period
Tinv = [19801001, 19951001] # Inversion period for historical forcings
# Tinv = [19891001, 19991001] # PUB/PUR period
Nfea = 12 # number of HBV parameters. 12:original HBV; 13:includes the added dynamic ET para when setting ETMod=True
BUFFTIME = 365 # for each training sample, to use BUFFTIME days to warm up the states.
routing = True # Whether to use the routing module for simulated runoff
Nmul = 16 # Multi-component model. How many parallel HBV components to use. 1 means the original HBV.
comprout = False # True is doing routing for each component
compwts = False # True is using weighted average for components; False is the simple mean
pcorr = None # or a list to give the range of precip correction

flow_regime = 0  # 1 is high flow expert.

if TDOpt is True:
    # Below options are only for running models with dynamic parameters
    tdRep = [1, 13] # When using dynamic parameters, this list defines which parameters to set as dynamic
    tdRepS = [str(ix) for ix in tdRep]
    # ETMod: if True, use the added shape parameter (index 13) for ET. Default as False.
    # Must set below ETMod as True and Nfea=13 when including 13 index in above tdRep list for dynamic parameters
    # If 13 not in tdRep list, set below ETMod=False and Nfea=12 to use the original HBV without ET shape para
    ETMod = True
    Nfea = 13 # should be 13 when setting ETMod=True. 12 when ETMod=False
    dydrop = 0.0 # dropout possibility for those dynamic parameters: 0.0 always dynamic; 1.0 always static
    staind = -1 # which time step to use from the learned para time series for those static parameters
    TDN = '/TDTestforc/'+'TD'+"_".join(tdRepS) +'/'
else:
    TDN = '/Testforc/'

# Define root directory of database and output
# Modify these based on your own location of CAMELS dataset
# Following the data download instruction in README file, you should organize the folders like
# 'your/path/to/Camels/basin_timeseries_v1p2_metForcing_obsFlow' and 'your/path/to/Camels/camels_attributes_v2.0'
# Then 'rootDatabase' here should be 'your/path/to/Camels';
# 'rootOut' is the root dir where you save the trained model
    


# CAMELS dataset root directory
sysroot = '/Users/leoglonz/Desktop/water/'
rootDatabase = os.path.join(os.path.sep, sysroot, 'data', 'Camels')    # CAMELS dataset root directory
# rootDatabase = os.path.join(os.path.sep, 'data', 'kas7897', 'dPLHBVrelease')  # CAMELS dataset root directory
camels.initcamels(rootDatabase, forType=forType)  # initialize camels module-scope variables in camels.py (dirDB, gageDict) to read basin info

# rootOut = os.path.join(os.path.sep, 'data', 'rnnStreamflow')  # Model output root directory
rootOut = os.path.join(os.path.sep, sysroot, 'data', 'model_runs', 'hydro_multimodel_results', 'multimodels', '671_sites_dp', 'mm_ensemble_sigmoid')  # Model output root directory



## If you have a checkpoint file for a previous run, set the path to directory here. 
# saved epoch is the epoch corresponding to the checkpoint. 
saved_epoch = 6
checkpoint_file = None #'/Users/leoglonz/Desktop/water/data/model_runs/rnnStreamflow/CAMELSDemo/dPLHBV/ALL/TDTestforc/TD1_13/daymet/BuffOpt0/RMSE_para0.25/111111/Fold1/T_19801001_19951001_BS_100_HS_256_RHO_365_NF_13_Buff_365_Mul_16/'

## set up different data loadings for ALL, PUB, PUR
testfoldInd = 1
# Which fold to hold out for PUB (10 folds, from 1 to 10) and PUR (7 folds, from 1 to 7).
# It doesn't matter when training on ALL basins (setting PUOpt=0), could always set testfoldInd=1 for this case.

# load CAMELS basin information
gageinfo = camels.gageDict
hucinfo = gageinfo['huc']
gageid = gageinfo['id']
gageidLst = gageid.tolist()

if PUOpt == 0: # training on all basins without spatial hold-out
    puN = 'ALL'
    TrainLS = gageidLst # all basins
    TrainInd = [gageidLst.index(j) for j in TrainLS]
    TestLS = gageidLst
    TestInd = [gageidLst.index(j) for j in TestLS]
    gageDic = {'TrainID':TrainLS, 'TestID':TestLS}

elif PUOpt == 1: # random hold out basins. hold out the fold set by testfoldInd
    puN = 'PUB'
    # load the PUB basin groups
    # randomly divide CAMELS basins into 10 groups and this file contains the basin ID for each group
    # located in splitPath
    splitPath = 'PUBsplitLst.txt'
    with open(splitPath, 'r') as fp:
        testIDLst=json.load(fp)
    # Generate training ID lists excluding the hold out fold
    TestLS = testIDLst[testfoldInd - 1]
    TestInd = [gageidLst.index(j) for j in TestLS]
    TrainLS = list(set(gageid.tolist()) - set(TestLS))
    TrainInd = [gageidLst.index(j) for j in TrainLS]
    gageDic = {'TrainID':TrainLS, 'TestID':TestLS}

elif PUOpt == 2:
    puN = 'PUR'
    # Divide CAMELS dataset into 7 continous PUR regions, as shown in Feng et al, 2021 GRL; 2022 HESSD
    # get the id list of each PUR region, save to list
    regionID = list()
    regionNum = list()
    # seven regions including different HUCs
    regionDivide = [ [1,2], [3,6], [4,5,7], [9,10], [8,11,12,13], [14,15,16,18], [17] ]
    for ii in range(len(regionDivide)):
        tempcomb = regionDivide[ii]
        tempregid = list()
        for ih in tempcomb:
            tempid = gageid[hucinfo==ih].tolist()
            tempregid = tempregid + tempid
        regionID.append(tempregid)
        regionNum.append(len(tempregid))

    iexp = testfoldInd - 1  #index
    TestLS = regionID[iexp] # basin ID list for testing, hold out for training
    TestInd = [gageidLst.index(j) for j in TestLS]
    TrainLS = list(set(gageid.tolist()) - set(TestLS)) # basin ID for training
    TrainInd = [gageidLst.index(j) for j in TrainLS]
    gageDic = {'TrainID': TrainLS, 'TestID': TestLS}


# apply buffOPt to solve the warm-up for the first year
if buffOpt ==2: # load more BUFFTIME data for the first year
    sd = utils.time.t2dt(Ttrain[0]) - dt.timedelta(days=BUFFTIME)
    sdint = int(sd.strftime("%Y%m%d"))
    TtrainLoad = [sdint, Ttrain[1]]
    TinvLoad = [sdint, Ttrain[1]]
else:
    TtrainLoad = Ttrain
    TinvLoad = Tinv

## prepare input data
## load camels dataset
if forType == 'daymet':
    varF = ['prcp', 'tmean']
    varFInv = ['prcp', 'tmean']
else:
    varF = ['prcp', 'tmax'] # For CAMELS maurer and nldas forcings, tmax is actually tmean
    varFInv = ['prcp', 'tmax']

# the attributes used to learn parameters
attrnewLst = [ 'p_mean','pet_mean','p_seasonality','frac_snow','aridity','high_prec_freq','high_prec_dur',
               'low_prec_freq','low_prec_dur', 'elev_mean', 'slope_mean', 'area_gages2', 'frac_forest', 'lai_max',
               'lai_diff', 'gvf_max', 'gvf_diff', 'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50',
               'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity',
               'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'geol_1st_class', 'glim_1st_class_frac',
               'geol_2nd_class', 'glim_2nd_class_frac', 'carbonate_rocks_frac', 'geol_porostiy', 'geol_permeability']

optData = default.optDataCamels # a default dictionary for logging, updated below
# Update the training period and variables
optData = default.update(optData, tRange=TtrainLoad, varT=varFInv, varC=attrnewLst, subset=TrainLS, forType=forType)

# for HBV model training inputs
dfTrain = camels.DataframeCamels(tRange=TtrainLoad, subset=TrainLS, forType=forType)
forcUN = dfTrain.getDataTs(varLst=varF, doNorm=False, rmNan=False, flow_regime=flow_regime)
obsUN = dfTrain.getDataObs(doNorm=False, rmNan=False, basinnorm=False, flow_regime=flow_regime)

# for dPL inversion data, inputs of gA
dfInv = camels.DataframeCamels(tRange=TinvLoad, subset=TrainLS, forType=forType)
forcInvUN = dfInv.getDataTs(varLst=varFInv, doNorm=False, rmNan=False, flow_regime=flow_regime)
attrsUN = dfInv.getDataConst(varLst=attrnewLst, doNorm=False, rmNan=False, flow_regime=flow_regime)

# Unit transformation, discharge obs from ft3/s to mm/day
areas = gageinfo['area'][TrainInd] # unit km2
temparea = np.tile(areas[:, None, None], (1, obsUN.shape[1],1))
obsUN = (obsUN * 0.0283168 * 3600 * 24) / (temparea * (10 ** 6)) * 10**3 # transform to mm/day

# load potential ET calculated by hargreaves method
varLstNL = ['PEVAP']
usgsIdLst = gageid
if forType == 'maurer':
    tPETRange = [19800101, 20090101]
else:
    tPETRange = [19800101, 20150101]
tPETLst = utils.time.tRange2Array(tPETRange)

# Modify this as the directory where you put PET
PETDir = rootDatabase + '/pet_harg/' + forType + '/'
ntime = len(tPETLst)
PETfull = np.empty([len(usgsIdLst), ntime, len(varLstNL)])
for k in range(len(usgsIdLst)):
    dataTemp = camels.readcsvGage(PETDir, usgsIdLst[k], varLstNL, ntime)
    PETfull[k, :, :] = dataTemp

TtrainLst = utils.time.tRange2Array(TtrainLoad)
TinvLst = utils.time.tRange2Array(TinvLoad)
C, ind1, ind2 = np.intersect1d(TtrainLst, tPETLst, return_indices=True)
PETUN = PETfull[:, ind2, :]
PETUN = PETUN[TrainInd, :, :] # select basins
C, ind1, ind2inv = np.intersect1d(TinvLst, tPETLst, return_indices=True)
PETInvUN = PETfull[:, ind2inv, :]
PETInvUN = PETInvUN[TrainInd, :, :]

# process data, do normalization and remove nan
series_inv = np.concatenate([forcInvUN, PETInvUN], axis=2)
seriesvarLst = varFInv + ['pet']
# calculate statistics for normalization and saved to a dictionary
statDict = camels.getStatDic(attrLst=attrnewLst, attrdata=attrsUN, seriesLst=seriesvarLst, seriesdata=series_inv, flow_regime=flow_regime)
# normalize data
attr_norm = camels.transNormbyDic(attrsUN, attrnewLst, statDict, toNorm=True, flow_regime=flow_regime)
attr_norm[np.isnan(attr_norm)] = 0.0
series_norm = camels.transNormbyDic(series_inv, seriesvarLst, statDict, toNorm=True, flow_regime=flow_regime)
series_norm[np.isnan(series_norm)] = 0.0

# prepare the inputs
zTrain = series_norm # used as the inputs for dPL inversion gA along with attributes
xTrain = np.concatenate([forcUN, PETUN], axis=2) # used as HBV forcing
xTrain[np.isnan(xTrain)] = 0.0

if buffOpt == 1: # repeat the first year warm up the first year itself
    zTrainIn = np.concatenate([zTrain[:,0:BUFFTIME,:], zTrain], axis=1)
    xTrainIn = np.concatenate([xTrain[:,0:BUFFTIME,:], xTrain], axis=1) # repeat forcing to warm up the first year
    yTrainIn = np.concatenate([obsUN[:,0:BUFFTIME,:], obsUN], axis=1)
else: # no repeat, original data, the first year data would only be used as warmup for the next following year
    zTrainIn = zTrain
    xTrainIn = xTrain
    yTrainIn = obsUN

forcTuple = (xTrainIn, zTrainIn)
attrs = attr_norm

## Train the model
# define loss function
alpha = 0.25 # a weight for RMSE loss to balance low and peak flow
optLoss = default.update(default.optLossComb, name='hydroDL.model.crit.RmseLossComb', weight=alpha)
lossFun = crit.RmseLossComb(alpha=alpha)

# define training options
optTrain = default.update(default.optTrainCamels, miniBatch=[BATCH_SIZE, RHO], nEpoch=EPOCH, saveEpoch=saveEPOCH)
# define output folder to save model results
exp_name = 'CAMELSDemo'
exp_disp = 'dPLHBV/' + puN + TDN + forType + '/BuffOpt'+str(buffOpt)+'/RMSE_para'+str(alpha)+'/' + str(randomseed) + \
           '/Fold' + str(testfoldInd)
exp_info = 'T_'+str(Ttrain[0])+'_'+str(Ttrain[1])+'_BS_'+str(BATCH_SIZE)+'_HS_'+str(HIDDENSIZE)\
           +'_RHO_'+str(RHO)+'_NF_'+str(Nfea)+'_Buff_'+str(BUFFTIME)+'_Mul_'+str(Nmul)
save_path = os.path.join(exp_name, exp_disp)
out = os.path.join(rootOut, save_path, exp_info) # output folder to save results
# define and load model
Ninv = zTrain.shape[-1] + attrs.shape[-1]

loading package hydroDL
daymet tmean was used!
read usgs streamflow 4.619056224822998
read usgs streamflow 4.002650022506714
read usgs streamflow 5.169572114944458
daymet tmean was used!
read usgs streamflow 4.245026111602783
read usgs streamflow 4.149972915649414


In [67]:
statDict

{'prcp': [-1.0, 0.533597287717274, -0.5355190506932449, 0.6686731458008723],
 'tmean': [-3.6750000000000003, 23.87, 10.222146296879702, 10.520438079586562],
 'pet': [0.57028911, 5.60688872, 2.943688350438002, 1.8915542037135196],
 'p_mean': [1.55055167693361,
  4.64999041752225,
  3.257722061222179,
  1.4093900773162575],
 'pet_mean': [2.1470467761807,
  3.55741331964408,
  2.7873652197155474,
  0.5545586452738964],
 'p_seasonality': [-0.836194748180532,
  0.621476145395479,
  -0.04127946765143021,
  0.5270366120903908],
 'frac_snow': [0.0030982528891208,
  0.52956688772688,
  0.1776040592521807,
  0.2031358038480485],
 'aridity': [0.543704960792406,
  1.87330300304748,
  1.0565155490835534,
  0.6149898629889541],
 'high_prec_freq': [13.75, 25.85, 20.930998509687036, 4.541832424799546],
 'high_prec_dur': [1.16627078384798,
  1.6517571884984,
  1.3498370635820642,
  0.1910832155245367],
 'low_prec_freq': [204.35, 300.45, 254.64731743666167, 35.089201003742616],
 'low_prec_dur': [3.45440

In [62]:
series_norm = camels.transNormbyDic(series_inv, seriesvarLst, statDict, toNorm=True, flow_regime=flow_regime)


array([[[ 1.20416251, -0.39372375, -0.8220996 ],
        [ 1.30078058,  0.02926244, -0.56379756],
        [ 1.49950266,  0.15330671, -0.66241314],
        ...,
        [ 1.17727655, -0.29629434, -0.69075544],
        [-0.69463078, -0.51634221, -0.50557539],
        [-0.69463078, -0.1418331 , -0.22237922]],

       [[-0.69463078,  0.12716711, -0.47018761],
        [-0.69463078,  0.25643929, -0.47469026],
        [ 1.40663177,  0.38000829, -0.63604434],
        ...,
        [ 1.29264226,  0.08867061, -0.42786305],
        [-0.69463078, -0.33621664, -0.39681128],
        [-0.69463078, -0.24163883, -0.31861828]],

       [[ 1.12832949, -0.10571293, -0.66361204],
        [ 1.05319639,  0.0682342 , -0.59036572],
        [ 1.57072137,  0.32392698, -0.59902711],
        ...,
        [ 1.31296095,  0.0078755 , -0.44710892],
        [-0.69463078, -0.48117258, -0.49237815],
        [-0.69463078, -0.29581908, -0.32006159]],

       ...,

       [[-0.69463078,  0.60338302, -0.00402899],
        [-0

In [60]:
xTrain

array([[[ 3.1      ,  6.08     ,  1.3886424],
        [ 4.24     , 10.53     ,  1.8772347],
        [ 8.02     , 11.835    ,  1.690698 ],
        ...,
        [ 2.84     ,  7.105    ,  1.637087 ],
        [ 0.       ,  4.79     ,  1.9873651],
        [ 0.       ,  8.73     ,  2.523046 ]],

       [[ 0.       , 11.56     ,  2.054303 ],
        [ 0.       , 12.92     ,  2.045786 ],
        [ 5.96     , 14.22     ,  1.740576 ],
        ...,
        [ 4.13     , 11.155    ,  2.1343622],
        [ 0.       ,  6.685    ,  2.1930983],
        [ 0.       ,  7.68     ,  2.3410046]],

       [[ 2.42     ,  9.11     ,  1.6884302],
        [ 1.89     , 10.94     ,  1.8269796],
        [10.06     , 13.63     ,  1.8105961],
        ...,
        [ 4.41     , 10.305    ,  2.0979576],
        [ 0.       ,  5.16     ,  2.0123284],
        [ 0.       ,  7.11     ,  2.3382745]],

       ...,

       [[ 0.       , 16.57     ,  2.9360673],
        [ 0.       , 19.175    ,  3.7661178],
        [ 0.       , 1

In [ ]:
if TDOpt is False:
    # model with all static parameters
    # model = rnn_new.MultiInv_HBVModel(ninv=Ninv, nfea=Nfea, nmul=Nmul, hiddeninv=HIDDENSIZE, inittime=BUFFTIME,
    #                                   routOpt = routing, comprout=comprout, compwts=compwts, pcorr=pcorr, multiforcing=multiforcing, prcp_datatypes=len(forType))
    model = rnn.prcp_weights(ninv=Ninv, hiddeninv=HIDDENSIZE, prcp_datatypes=len(forType))
    # dict only for logging
    optModel = OrderedDict(name='LSTM-dPLHBV', nx=Ninv, nfea=Nfea, nmul=Nmul, hiddenSize=HIDDENSIZE, doReLU=True,
                           Tinv=Tinv, Trainbuff=BUFFTIME, routOpt=routing, comprout=comprout, compwts=compwts,
                           pcorr=pcorr, buffOpt=buffOpt, TDOpt=TDOpt)
else:
    # model with some dynamic parameters
    # model = rnn_new.MultiInv_HBVTDModel(ninv=Ninv, nfea=Nfea, nmul=Nmul, hiddeninv=HIDDENSIZE, inittime=BUFFTIME,
    #                               routOpt=routing, comprout=comprout, compwts=compwts, staind=staind, tdlst=tdRep,
    #                               dydrop=dydrop, ETMod=ETMod, multiforcing=multiforcing, prcp_datatypes=len(forType))
    model = rnn.prcp_weights(ninv=Ninv, hiddeninv=HIDDENSIZE, prcp_datatypes=len(forType))

    # dict only for logging
    optModel = OrderedDict(name='LSTM-dPLHBV', nx=Ninv, nfea=Nfea, nmul=Nmul, hiddenSize=HIDDENSIZE, doReLU=True,
                           Tinv=Tinv, Trainbuff=BUFFTIME, routOpt=routing, comprout=comprout, compwts=compwts,
                           pcorr=pcorr, staind=staind, tdlst=tdRep, dydrop=dydrop,buffOpt=buffOpt, TDOpt=TDOpt, ETMod=ETMod)

In [ ]:
import math
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.autograd as autograd

from hydroDL.model.dropout import DropMask, createMask



class CudnnLstm(nn.Module):
    def __init__(self, *, inputSize, hiddenSize, dr=0.5, drMethod="drW", gpu=0, seed=42):
        super(CudnnLstm, self).__init__()
        self.name = 'CudnnLstm'
        self.inputSize = inputSize
        self.hiddenSize = hiddenSize
        self.dr = dr

        self.w_ih = Parameter(torch.Tensor(hiddenSize * 4, inputSize))
        self.w_hh = Parameter(torch.Tensor(hiddenSize * 4, hiddenSize))
        self.b_ih = Parameter(torch.Tensor(hiddenSize * 4))
        self.b_hh = Parameter(torch.Tensor(hiddenSize * 4))
        self._all_weights = [['w_ih', 'w_hh', 'b_ih', 'b_hh']]
        self.cuda()
        self.seed = seed
        self.is_legacy = True

        self.reset_mask()
        self.reset_parameters()

    def _apply(self, fn):
        ret = super(CudnnLstm, self)._apply(fn)
        return ret

    def __setstate__(self, d):
        super(CudnnLstm, self).__setstate__(d)
        self.__dict__.setdefault("_data_ptrs", [])
        if "all_weights" in d:
            self._all_weights = d["all_weights"]
        if isinstance(self._all_weights[0][0], str):
            return
        self._all_weights = [["w_ih", "w_hh", "b_ih", "b_hh"]]

    def reset_mask(self):
        self.maskW_ih = createMask(self.w_ih, self.dr, self.seed)
        self.maskW_hh = createMask(self.w_hh, self.dr, self.seed)

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hiddenSize)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, input, hx=None, cx=None, doDropMC=False, dropoutFalse=False):
        # dropoutFalse: it will ensure doDrop is false, unless doDropMC is true
        if dropoutFalse and (not doDropMC):
            doDrop = False
        elif self.dr > 0 and (doDropMC is True or self.training is True):
            doDrop = True
        else:
            doDrop = False

        batchSize = input.size(1)
            
        if hx is None:
            hx = input.new_zeros(1, batchSize, self.hiddenSize, requires_grad=False)
        if cx is None:
            cx = input.new_zeros(1, batchSize, self.hiddenSize, requires_grad=False)

        # cuDNN backend - disabled flat weight
        # handle = torch.backends.cudnn.get_handle()
        if doDrop is True:
            self.reset_mask()
            weight = [
                DropMask.apply(self.w_ih, self.maskW_ih, True),
                DropMask.apply(self.w_hh, self.maskW_hh, True), self.b_ih,
                self.b_hh
            ]
        else:
            weight = [self.w_ih, self.w_hh, self.b_ih, self.b_hh]

        # output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
        # input, weight, 4, None, hx, cx, torch.backends.cudnn.CUDNN_LSTM,
        # self.hiddenSize, 1, False, 0, self.training, False, (), None)
        if torch.__version__ < "1.8":
            output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
                input,
                weight,
                4,
                None,
                hx,
                cx,
                2,  # 2 means LSTM
                self.hiddenSize,
                1,
                False,
                0,
                self.training,
                False,
                (),
                None,
            )
        else:
            output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
                input,
                weight,
                4,
                None,
                hx,
                cx,
                2,  # 2 means LSTM
                self.hiddenSize,
                0,
                1,
                False,
                0,
                self.training,
                False,
                (),
                None,
            )
        return output, (hy, cy)

    @property
    def all_weights(self):
        return [
            [getattr(self, weight) for weight in weights]
            for weights in self._all_weights
        ]
    


class CudnnLstmModel(torch.nn.Module):
    def __init__(self, *, nx, ny, hiddenSize, dr=0.5, warmUpDay=None):
        super(CudnnLstmModel, self).__init__()
        self.name = 'CudnnLstmModel'
        self.nx = nx
        self.ny = ny
        self.hiddenSize = hiddenSize
        self.ct = 0
        self.nLayer = 1
        self.linearIn = torch.nn.Linear(nx, hiddenSize)

        self.lstm = CudnnLstm(
            inputSize=hiddenSize, hiddenSize=hiddenSize, dr=dr
        )
        self.linearOut = torch.nn.Linear(hiddenSize, ny)
        self.gpu = 1
        self.is_legacy = True
        # self.drtest = torch.nn.Dropout(p=0.4)
        self.warmUpDay = warmUpDay

    def forward(self, x, doDropMC=False, dropoutFalse=False):
        """
        :param inputs: a dictionary of input data (x and potentially z data)
        :param doDropMC:
        :param dropoutFalse:
        :return:
        """
        # if not self.warmUpDay is None:
        #     x, warmUpDay = self.extend_day(x, warm_up_day=self.warmUpDay)

        x0 = F.relu(self.linearIn(x))

        outLSTM, (hn, cn) = self.lstm(
            x0, doDropMC=doDropMC, dropoutFalse=dropoutFalse
        )
        # outLSTMdr = self.drtest(outLSTM)
        out = self.linearOut(outLSTM)
        
        # if not self.warmUpDay is None:
        #     out = self.reduce_day(out, warm_up_day=self.warmUpDay)

        return out
    
    def extend_day(self, x, warm_up_day):
        x_num_day = x.shape[0]
        warm_up_day = min(x_num_day, warm_up_day)
        x_select = x[:warm_up_day, :, :]
        x = torch.cat([x_select, x], dim=0)
        return x, warm_up_day

    def reduce_day(self, x, warm_up_day):
        x = x[warm_up_day:,:,:]
        return x


class RangeBoundLoss(nn.Module):
    """limit parameters from going out of range"""
    def __init__(self, lb, ub):
        super(RangeBoundLoss, self).__init__()
        self.lb = torch.tensor(lb).cuda()
        self.ub = torch.tensor(ub).cuda()
        # self.factor = torch.tensor(factor).cuda()

    def forward(self, params, factor):
        factor = torch.tensor(factor).cuda()
        loss = 0
        for i in range(len(params)):
            lb = self.lb[i]
            ub = self.ub[i]
            upper_bound_loss = factor * torch.relu(params[i] - ub).mean()
            lower_bound_loss = factor * torch.relu(lb - params[i]).mean()
            loss = loss + upper_bound_loss + lower_bound_loss
        return loss

In [ ]:
class EnsembleWeights(torch.nn.Module):
    def __init__(self, *, ninv, hiddeninv, drinv=0.5, nmodels=1):
        super(EnsembleWeights, self).__init__()
        self.name = 'EnsembleWeights'
        self.ninv = ninv
        self.nmodels = nmodels

        ntp = nmodels
        self.hiddeninv = hiddeninv

        self.lstminv = CudnnLstmModel(
            nx=ninv, ny=ntp, hiddenSize=hiddeninv, dr=drinv)
        lb_prcp = [0.95]
        ub_prcp = [1.05]
        self.RangeBoundLoss = RangeBoundLoss(lb=lb_prcp, ub=ub_prcp)

    def forward(self, x, z, prcp_loss_factor):
        z.requires_grad = True

        wghts = self.lstminv(z)
        ntstep = wghts.shape[0]
        ngage = wghts.shape[1]
        wghts_scaled = torch.sigmoid(wghts)  # Change weighting method here.

        prcp_wavg = torch.zeros((ntstep, ngage), requires_grad=True, dtype=torch.float32).cuda()
        prcp_wghts_sum = torch.sum(wghts_scaled, dim=2)
        range_bound_loss_prcp = self.RangeBoundLoss([prcp_wghts_sum], factor=prcp_loss_factor)

        for para in range(wghts.shape[2]):
            prcp_wavg = prcp_wavg + wghts_scaled[:, :, para] * x[:, :, para]

        x_new = torch.empty((ntstep, ngage, 3), requires_grad=True, dtype=torch.float32).cuda()
        x_new[:, :, 0] = prcp_wavg
        x_new[:, :, 1] = x[:, :, self.prcp_datatypes]
        x_new[:, :, 2] = x[:, :, -1]

        # For gradient analysis.
        # grad_daymet = autograd.grad(outputs=wghts_scaled[:, :, 0], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 0]), retain_graph=True)[0]
        # grad_maurer = autograd.grad(outputs=wghts_scaled[:, :, 1], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 1]), retain_graph=True)[0]
        # grad_nldas = autograd.grad(outputs=wghts_scaled[:, :, 2], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 2]), retain_graph=True)[0]

        # return x_new, range_bound_loss_prcp, wghts_scaled, grad_daymet, grad_maurer, grad_nldas
        return x_new, range_bound_loss_prcp, wghts_scaled

In [ ]:
from typing import ValuesView
from tqdm import tqdm
import time


def randomIndex(ngrid, nt, dimSubset, bufftime=0):
    batchSize, rho = dimSubset
    iGrid = np.random.randint(0, ngrid, [batchSize])
    iT = np.random.randint(0 + bufftime, nt - rho, [batchSize])
    return iGrid, iT



def selectSubset(x, iGrid, iT, rho, *, c=None, tupleOut=False, LCopt=False, bufftime=0):
    nx = x.shape[-1]
    nt = x.shape[1]
    if x.shape[0] == len(iGrid):   #hack
        iGrid = np.arange(0,len(iGrid))  # hack
    if nt <= rho:
        iT.fill(0)

    batchSize = iGrid.shape[0]
    if iT is not None:
        # batchSize = iGrid.shape[0]
        xTensor = torch.zeros([rho+bufftime, batchSize, nx], requires_grad=False)
        for k in range(batchSize):
            temp = x[iGrid[k]:iGrid[k] + 1, np.arange(iT[k]-bufftime, iT[k] + rho), :]
            xTensor[:, k:k + 1, :] = torch.from_numpy(np.swapaxes(temp, 1, 0))
    else:
        if LCopt is True:
            # used for local calibration kernel: FDC, SMAP...
            if len(x.shape) == 2:
                # Used for local calibration kernel as FDC
                # x = Ngrid * Ntime
                xTensor = torch.from_numpy(x[iGrid, :]).float()
            elif len(x.shape) == 3:
                # used for LC-SMAP x=Ngrid*Ntime*Nvar
                xTensor = torch.from_numpy(np.swapaxes(x[iGrid, :, :], 1, 2)).float()
        else:
            # Used for rho equal to the whole length of time series
            xTensor = torch.from_numpy(np.swapaxes(x[iGrid, :, :], 1, 0)).float()
            rho = xTensor.shape[0]
    if c is not None:
        nc = c.shape[-1]
        temp = np.repeat(
            np.reshape(c[iGrid, :], [batchSize, 1, nc]), rho+bufftime, axis=1)
        cTensor = torch.from_numpy(np.swapaxes(temp, 1, 0)).float()

        if (tupleOut):
            if torch.cuda.is_available():
                xTensor = xTensor.cuda()
                cTensor = cTensor.cuda()
            out = (xTensor, cTensor)
        else:
            out = torch.cat((xTensor, cTensor), 2)
    else:
        out = xTensor

    if torch.cuda.is_available() and type(out) is not tuple:
        out = out.cuda()
    return out




def trainModel(model,
                loaded_hbv,
                x,
                y,
                c,
                lossFun,
                *,
                nEpoch=500,
                miniBatch=[100, 30],
                saveEpoch=100,
                saveFolder=None,
                mode='seq2seq',
                bufftime=0,
                prcp_loss_factor = 15,
                smooth_loss_factor = 0,
                ):
    """
    inputs:
    x - input
    z - normalized x input
    y - target, or observed values
    c - constant input, attributes
    """

    batchSize, rho = miniBatch
    if type(x) is tuple or type(x) is list:
        x, z = x 

    ngrid, nt, nx = x.shape  # ngrid= # basins, nt= # timesteps, nx= # attributes

    if c is not None:
        nx = nx + c.shape[-1]
    if batchSize >= ngrid:
        # Cannot have more batches than there are basins.
        batchSize = ngrid

    nIterEp = int(
        np.ceil(np.log(0.01) / np.log(1 - batchSize * rho / ngrid / (nt-bufftime)))
        )
    if hasattr(model, 'ctRm'):
        if model.ctRm is True:
            nIterEp = int(
                np.ceil(
                    np.log(0.01) / np.log(1 - batchSize *
                                          (rho - model.ct) / ngrid / (nt-bufftime))))

    if torch.cuda.is_available():
        lossFun = lossFun.cuda()
        model = model.cuda()

    optim = torch.optim.Adadelta(list(model.parameters()))
    model.zero_grad()

    # Save file.
    if saveFolder is not None:
        os.makedirs(saveFolder, exist_ok=True)
        runFile = os.path.join(saveFolder, 'run.csv')
        rf = open(runFile, 'w+')

    for iEpoch in range(1, nEpoch + 1):
        lossEp = 0
        loss_prcp_Ep = 0
        loss_sf_Ep = 0
        # loss_smooth_Ep = 0

        t0 = time.time()
        prog_str = "Epoch " + str(iEpoch) + "/" + str(nEpoch)

        for iIter in tqdm(range(0, nIterEp), desc=prog_str, leave=False):
            # training iterations
            if type(model) in [EnsembleWeights]:
                iGrid, iT = randomIndex(ngrid, nt, [batchSize, rho], bufftime=bufftime)
                
                xTrain = selectSubset(x, iGrid, iT, rho, bufftime=bufftime)
                yTrain = selectSubset(y, iGrid, iT, rho)
                zTrain = selectSubset(z, iGrid, iT, rho)
                    
                # calculate loss and weights `wt`.
                xP, prcp_loss, prcp_weights = model(xTrain, zTrain, prcp_loss_factor)
                yP = np.sum(xTrain * prcp_weights, axis=2)
                
            else:
                Exception('unknown model')

            # Consider the buff time for initialization.
            if bufftime > 0:
                yP = yP[bufftime:,:,:]

            # get loss
            if type(lossFun) in [crit.NSELossBatch, crit.NSESqrtLossBatch]:
                loss_sf = lossFun(yP, yTrain, iGrid)
                loss =  loss_sf + prcp_loss
            else:
                loss_sf = lossFun(yP, yTrain)
                loss = loss_sf + prcp_loss

            loss.backward()
            optim.step()
            optim.zero_grad()
            lossEp = lossEp + loss.item()

            try:
                loss_prcp_Ep = loss_prcp_Ep + prcp_loss.item()
            except:
                pass
            
            loss_sf_Ep = loss_sf_Ep + loss_sf.item()

            # if iIter % 100 == 0:
            #     print('Iter {} of {}: Loss {:.3f}'.format(iIter, nIterEp, loss.item()))

        # print loss
        lossEp = lossEp / nIterEp
        loss_sf_Ep = loss_sf_Ep / nIterEp
        loss_prcp_Ep = loss_prcp_Ep / nIterEp

        logStr = 'Epoch {} Loss {:.3f}, Streamflow Loss {:.3f}, Precipitation Loss {:.3f}, time {:.2f}'.format(
            iEpoch, lossEp, loss_sf_Ep, loss_prcp_Ep,
            time.time() - t0)
        print(logStr)

        # Save model and loss.
        if saveFolder is not None:
            rf.write(logStr + '\n')
            if iEpoch % saveEpoch == 0:
                # save model
                modelFile = os.path.join(saveFolder,
                                         'model_Ep' + str(iEpoch) + '.pt')
                torch.save(model, modelFile)

    if saveFolder is not None:
        rf.close()

    return model


In [ ]:


trainedModel = train.trainModel(
    model,
    forcTuple,
    yTrainIn,
    attrs,
    lossFun,
    nEpoch=EPOCH,
    miniBatch=[BATCH_SIZE, RHO],
    saveEpoch=saveEPOCH,
    saveFolder=out,
    bufftime=BUFFTIME,
    prcp_loss_factor=prcp_loss_factor,
    smooth_loss_factor=smooth_loss_factor)